<h1>SJ Training fNIRS Data Analysis</h1>
Written by Ansley Kunnath
Updated July 29, 2024
Python v3.9.12

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pandas as pd
import glob
import csv
from scipy.stats import ttest_rel, zscore
import os.path as op
import statsmodels.formula.api as smf
import mne_nirs.preprocessing
import mne_nirs.statistics
import mne_nirs.utils
import mne
from mne.preprocessing.nirs import tddr
from nilearn.glm.first_level import make_first_level_design_matrix  
from mne_nirs.channels import get_long_channels

# Open plots in new window
%matplotlib qt 

print("All done!") 

In [ ]:
# Locate files

subjects = ('201 202 203 204 205 206 207 208 209 212 213 214 215 216 217 218 219 221 223 224 225 226 228 229 230 231 232 233 234').split() #

# Mapping of subjects to groups
subject_to_group = {
    201: "trained",
    202: "control",
    203: "trained",
    204: "control",
    205: "control",
    206: "control",
    207: "trained",
    208: "control",
    209: "control",
    212: "trained",
    213: "trained",
    214: "trained",
    215: "control",
    216: "trained",
    217: "control",
    218: "control",
    219: "trained",
#    220: "control", #sampling rate = 2.40 Hz instead of 4.8
    221: "trained",
    223: "trained",
    224: "control",
    225: "trained",
    226: "control",
    228: "trained",
    229: "control",
    230: "trained",
    231: "trained",
    232: "control",
    233: "trained",
    234: "control",
}

sfreq = 4.807692
conditions = ('A', 'V', 'AV', 'W')
groups = ('trained','control')
days = ('1', '3')
runs = (1, 2)

condition_colors = dict(  # https://personal.sron.nl/~pault/data/colourschemes.pdf
    A='#4477AA',  # sblue
    AV='#CCBB44',  # yellow
    V='#EE7733',  # orange
    W='#AA3377',  # purple
)
exp_name = 'av'
duration = 1.8
design = 'event'
plot_subject = '205'
plot_day = 1
plot_run = 1
beh_title, beh_idx = 'AV', 0
filt_kwargs = dict(
    l_freq=0.01,
    h_freq=0.2) 
run_h = True  # regenerate HbO/HbR
n_jobs = 4  # for GLM

# SET FOLDER LOCATIONS
# I save the output files outside of the folder that's uploaded to Github
current_directory = %pwd
os.chdir(current_directory)

raw_path = '../../data'
proc_path = '../../processed'
results_path = '../../results'
subjects_dir = '../../subjects'
os.makedirs(proc_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)
os.makedirs(subjects_dir, exist_ok=True)
#mne.datasets.fetch_fsaverage(subjects_dir=subjects_dir, verbose=True)

use = None
all_sci = list()
plt.rcParams['axes.titlesize'] = 8
plt.rcParams['axes.labelsize'] = 8
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
got_bad = 0
got_total = 0

# Prep making bad channels report
bad_channels_filename = op.join(results_path, 'bad_channels_report.csv')
with open(bad_channels_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Subject', 'Day', 'Run','Percent Bad'])

def normalize_channel_names(channels_set):
    return {name.split()[0] for name in channels_set}

def add_bad_channel_entry(subject, day, run, percentage_bad):
    with open(bad_channels_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([subject, day, run, f'{percentage_bad:.2f}%'])

def piecewise_linear_detrend(raw, sfreq, segment_length):
    data = raw.get_data()
    n_channels, n_samples = data.shape
    segment_samples = int(segment_length * sfreq)    
    for ch in range(n_channels):
        for start in range(0, n_samples, segment_samples):
            end = min(start + segment_samples, n_samples)
            segment = data[ch, start:end]
            time = np.arange(start, end)
            p = np.polyfit(time, segment, 1)
            trend = np.polyval(p, time)
            data[ch, start:end] -= trend    
    raw._data = data
    return raw

def preprocess_fnirs_data(raw_data, proc_path, base):
    raw_od = mne.preprocessing.nirs.optical_density(raw_data, verbose='error')

    # Identify bad channels
    peaks = np.ptp(raw_od.get_data('fnirs'), axis=-1)
    flat_names = [raw_od.ch_names[f].split(' ')[0] for f in np.where(peaks < 0.001)[0]]
    sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
    sci_mask = (sci < 0.25)
    got = np.where(sci_mask)[0]
    percentage_bad = (len(got) / len(raw_od.ch_names)) * 100
    print(f'    Run {base}')

    # Assign bads
    assert raw_od.info['bads'] == []
    bads = set(raw_od.ch_names[pick] for pick in got)
    bads = bads | set(ch_name for ch_name in raw_od.ch_names if ch_name.split(' ')[0] in flat_names)
    bads = sorted(bads)

    # Further preprocessing
    raw_tddr = tddr(raw_od) # DON'T TDDR?
    raw_tddr_bp = raw_tddr.copy().filter(**filt_kwargs) # DON'T BANDPASS FILTER?
    #raw_tddr_bp = raw_od.copy() # Alternative Code
    raw_tddr_bp.info['bads'] = bads
    picks = mne.pick_types(raw_tddr_bp.info, fnirs=True)
    peaks = np.ptp(raw_tddr_bp.get_data(picks), axis=-1)
    assert (peaks > 1e-5).all()
    raw_tddr_bp.info['bads'] = [] 
    raw_h = mne.preprocessing.nirs.beer_lambert_law(raw_tddr_bp, 6.)

    # Normalize and verify bad channels
    h_bads = [ch_name for ch_name in raw_h.ch_names if ch_name.split(' ')[0] in set(bad.split(' ')[0] for bad in bads)]
    set_bads = set(bads)
    set_h_bads = set(h_bads)
    normalized_bads = normalize_channel_names(set_bads)
    normalized_h_bads = normalize_channel_names(set_h_bads)
    assert normalized_bads == normalized_h_bads
    raw_h.info['bads'] = h_bads
    raw_h.info._check_consistency()

    # Further verification
    picks = mne.pick_types(raw_h.info, fnirs=True)
    peaks = np.ptp(raw_h.get_data(picks), axis=-1)
    assert (peaks > 1e-9).all()

    # Interpolate bad channels
    raw_h_interp = raw_h.copy().interpolate_bads(reset_bads=True, method=dict(fnirs='nearest'))
    raw_h_interp.save(op.join(proc_path, f'{base}_hbo_raw.fif'), overwrite=True)
    assert len(raw_h.ch_names) == len(raw_h_interp.ch_names)

    return raw_h_interp, percentage_bad, bads

# Sanity check for subjects
subjects_check = {int(subject) for subject in subjects}
subject_to_group_check = set(subject_to_group.keys())
if subjects_check == subject_to_group_check:
    print("All done!") 
    print("N=" + str(len(subjects)))
    del subjects_check
    del subject_to_group_check
else:
    print("Error loading subject info") 

In [ ]:
# Remove patients with over 30% bad channels on average across days and runs

subjects_to_remove = ['202', '203', '204', '206', '214', '221', '223', '226', '233'] #201, 209?

# Initialize counters for each group
removed_trained = 0
removed_control = 0

remaining_trained = 0
remaining_control = 0

# Count and remove the subjects
for subject in subjects_to_remove:
    subject_int = int(subject)  # Convert to integer for dictionary key comparison
    if subject_int in subject_to_group:
        # Increment the appropriate counter based on the group of the subject
        if subject_to_group[subject_int] == "trained":
            removed_trained += 1
        elif subject_to_group[subject_int] == "control":
            removed_control += 1
        # Remove the subject from the dictionary
        subject_to_group.pop(subject_int, None)

# Update the subjects list after counting the removed subjects
subjects = [subject for subject in subjects if subject not in subjects_to_remove]

for group in subject_to_group.values():
    if group == "trained":
        remaining_trained += 1
    elif group == "control":
        remaining_control += 1

# Output the results
print(f'Removed {removed_trained} trained subjects.')
print(f'Removed {removed_control} control subjects.')
print(f'Remaining trained subjects: {remaining_trained}')
print(f'Remaining control subjects: {remaining_control}')


In [31]:
###############################################################################
# Load participant data

#subjects = ('201 202').split() #for testing

for subject in subjects:
    for day in days:
        for run in runs:
            group = subject_to_group.get(int(subject), "unknown")
            root1 = f'Day{day}'
            root2 = f'{subject}_{day}'
            root3 = f'*-*-*_{run:03d}'
            fname_base = op.join(raw_path, root1, root2, root3)
            fname = glob.glob(fname_base)
            base = f'{subject}_{day}_{run:03d}'
            base_pr = base.ljust(20)
            # Save the plot subject
            #if not op.isfile(op.join(proc_path, f'{base}_hbo_raw.fif')):
            raw_intensity = mne.io.read_raw_nirx(fname[0])
            long_chans = get_long_channels(raw_intensity)
            raw_h_long, percentage_bad_long, bads_long = preprocess_fnirs_data(long_chans, proc_path, base + '_long')
            add_bad_channel_entry(subject, day, run, percentage_bad_long)
            del raw_intensity
            if (op.isfile(op.join(proc_path, f'{base}_hbo_raw.fif')) and subject == plot_subject and run == plot_run):
                fname2 = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
                use = mne.io.read_raw_fif(fname2)
                events, _ = mne.events_from_annotations(use)
                ch_names = [ch_name.rstrip(' hbo') for ch_name in use.ch_names]
                info = use.info
                print("Saved plot file!")

print("All done!") 


    Run 205_1_001_long
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 0.20 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.20 Hz)
- Filter length: 1587 samples (330.096 sec)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s


Overwriting existing file.
Writing /Users/ansle/Documents/GitHub/fnirs-analysis/GithubCode/fnirs-analysis/../../processed/205_1_001_long_hbo_raw.fif


[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.3s
/var/folders/dv/s4k5_3r51ml3q36bgbcdsgbr0000gn/T/ipykernel_42700/1216949861.py:161: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_h_interp = raw_h.copy().interpolate_bads(reset_bads=True, method=dict(fnirs='nearest'))


Closing /Users/ansle/Documents/GitHub/fnirs-analysis/GithubCode/fnirs-analysis/../../processed/205_1_001_long_hbo_raw.fif
[done]
Opening raw data file ../../processed/205_1_001_hbo_raw.fif...
    Range : 0 ... 4171 =      0.000 ...   867.568 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Saved plot file!
Loading ../../data/Day1/205_1/2023-10-13_002
Reading 0 ... 4933  =      0.000 ...  1026.064 secs...
    Run 205_1_002_long
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 0.20 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.20 Hz)
- Filter length: 

...

Closing /Users/ansle/Documents/GitHub/fnirs-analysis/GithubCode/fnirs-analysis/../../processed/234_1_002_long_hbo_raw.fif
[done]
Loading ../../data/Day3/234_3/2024-07-04_001
Reading 0 ... 3883  =      0.000 ...   807.664 secs...
    Run 234_3_001_long
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 0.20 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.20 Hz)
- Filter length: 1587 samples (330.096 sec)

Overwriting existing file.
Writing /Users/ansle/Documents/GitHub/fnirs-analysis/GithubCode/fnirs-analysis/../../processed/234_3_001_long_hbo_raw.fif
Closing /Users/ansle/Documents/G

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
/var/folders/dv/s4k5_3r51ml3q36bgbcdsgbr0000gn/T/ipykernel_42700/1216949861.py:161: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_h_interp = raw_h.copy().interpolate_bads(reset_bads=True, method=dict(fnirs='nearest'))


Reading 0 ... 3956  =      0.000 ...   822.848 secs...
    Run 234_3_002_long
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 0.20 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.20 Hz)
- Filter length: 1587 samples (330.096 sec)

Overwriting existing file.
Writing /Users/ansle/Documents/GitHub/fnirs-analysis/GithubCode/fnirs-analysis/../../processed/234_3_002_long_hbo_raw.fif
Closing /Users/ansle/Documents/GitHub/fnirs-analysis/GithubCode/fnirs-analysis/../../processed/234_3_002_long_hbo_raw.fif
[done]


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
/var/folders/dv/s4k5_3r51ml3q36bgbcdsgbr0000gn/T/ipykernel_42700/1216949861.py:161: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_h_interp = raw_h.copy().interpolate_bads(reset_bads=True, method=dict(fnirs='nearest'))


All done!


In [ ]:
# OPTIONAL: Plot bad channels
 
bad_channels_df = pd.read_csv(bad_channels_filename)
bad_channels_df['Percent Bad'] = bad_channels_df['Percent Bad'].str.rstrip('%').astype(float)
bad_channels_df['Subject'] = bad_channels_df['Subject'].astype(str)

# Sort the DataFrame by percentage_bad in ascending order
bad_channels_df = bad_channels_df.groupby('Subject')['Percent Bad'].mean().reset_index()
bad_channels_df = bad_channels_df.sort_values(by='Percent Bad')
bad_channels_df['Group'] = bad_channels_df['Subject'].map(lambda x: subject_to_group[int(x)])

# Sort the DataFrame by Percent Bad in ascending order
colors = bad_channels_df['Group'].map({'control': 'gray', 'trained': 'lightblue'})

# Plotting
plt.figure(figsize=(12, 6))
plt.bar(bad_channels_df['Subject'], bad_channels_df['Percent Bad'], color=colors)
plt.axhline(30, color='red', linestyle='--')  # Horizontal line at 30%
plt.xlabel('Subject', fontsize=14)
plt.ylabel('Percent Bad', fontsize=14)
plt.title('Percentage of Bad Channels per Participant', fontsize=14)
plt.ylim(0, 100)  
plt.yticks(np.arange(0, 101, 10), [f'{i}%' for i in np.arange(0, 101, 10)])

output_filename = op.join(results_path, 'bad_channels_report.png')
plt.savefig(output_filename, bbox_inches='tight')
plt.show()


In [33]:
###############################################################################
# Define make_design

sfreq = 4.807692

for subject in subjects:
    for day in days:
        for run in runs:
            fname = op.join(proc_path, f'{subject}_{day}_{run:03d}_long_hbo_raw.fif')
            raw_h = mne.io.read_raw_fif(fname)
            events, _ = mne.events_from_annotations(raw_h)
            #print(len(events))

print("All done!") 

Opening raw data file ../../processed/201_1_001_long_hbo_raw.fif...
    Range : 0 ... 3994 =      0.000 ...   830.752 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file ../../processed/201_1_002_long_hbo_raw.fif...
    Range : 0 ... 3838 =      0.000 ...   798.304 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file ../../processed/201_3_001_long_hbo_raw.fif...
    Range : 0 ... 3945 =      0.000 ...   820.560 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file ../../processed/201_3_002_long_hbo_raw.fif...
    Range : 0 ... 3945 =      0.000 ...   820.560 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file ../../processed/205_1_001_long_hbo_raw.fif...
    Range : 0 ... 4171 =      0.000 ...   867.568 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw dat

In [ ]:
#subjects = ('224 225 226 228 229 230 231 232 233 234').split() #


In [34]:
###############################################################################
# Run GLM analysis and epoching

sfreq = 4.807692050933838

subj_cha_list = []
for subject in subjects:
    group = subject_to_group.get(int(subject), "unknown")
    
    for day in days:
        for run in runs:
            fname_long = op.join(proc_path, f'{subject}_{day}_{run:03d}_long_hbo_raw.fif')
            raw_h_long = mne.io.read_raw_fif(fname_long)
            _, dm, _ = _make_design(raw_h_long, design, subject, run, day, group)
            glm_est = mne_nirs.statistics.run_glm(
                raw_h_long, dm, noise_model='ols', n_jobs=n_jobs)
            cha = glm_est.to_dataframe()
            cha['subject'] = subject
            cha['run'] = run
            cha['day'] = day
            cha['group'] = group
            subj_cha_list.append(cha)
            del raw_h_long
        print(f'***Finished processing subject {subject} day {day}.')

df_cha = pd.concat(subj_cha_list, ignore_index=True)
df_cha.reset_index(drop=True, inplace=True)
print("Finished processing all subjects.")

Opening raw data file ../../processed/201_1_001_long_hbo_raw.fif...
    Range : 0 ... 3994 =      0.000 ...   830.752 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file ../../processed/201_1_002_long_hbo_raw.fif...
    Range : 0 ... 3838 =      0.000 ...   798.304 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
***Finished processing subject 201 day 1.
Opening raw data file ../../processed/201_3_001_long_hbo_raw.fif...
    Range : 0 ... 3945 =      0.000 ...   820.560 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file ../../processed/201_3_002_long_hbo_raw.fif...
    Range : 0 ... 3945 =      0.000 ...   820.560 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
***Finished processing subject 201 day 3.
Opening raw data file ../../processed/205_1_001_long_hbo_raw.fif...
    Range : 0 ... 4171 =      0.000 ...   867.568 secs
Ready

In [35]:
def _make_design(raw_h_long, design, subject=None, run=None, day=None, group=None):
    annotations_to_remove = raw_h_long.annotations.description == '255.0'
    raw_h_long.annotations.delete(annotations_to_remove)
    events, _ = mne.events_from_annotations(raw_h_long)
    rows_to_remove = events[:, -1] == 1
    events = events[~rows_to_remove]
    
    # Mis-codings
    if len(events) == 101:
        events = events[1:]
        
    n_times = len(raw_h_long.times)
    stim = np.zeros((n_times, 4))
    events[:, 2] -= 1
    assert len(events) == 100, len(events)
    want = [0] + [25] * 4
    count = np.bincount(events[:, 2])
    assert np.array_equal(count, want), count
    assert events.shape == (100, 3), events.shape
    if design == 'block':
        events = events[0::5]
        duration = 20.
        assert np.array_equal(np.bincount(events[:, 2]), [0] + [5] * 4)
    else:
#        assert design == 'event'
        assert len(events) == 100
        duration = 1.8
        assert events.shape == (100, 3)
        events_r = events[:, 2].reshape(20, 5)
        assert (events_r == events_r[:, :1]).all()
        del events_r
        
    idx = (events[:, [0, 2]] - [0, 1]).T
    assert np.in1d(idx[1], np.arange(len(conditions))).all()
    stim[tuple(idx)] = 1
    
    # assert raw_h.info['sfreq'] == sfreq  # necessary for below logic to work
    n_block = int(np.ceil(duration * sfreq))
    stim = signal.fftconvolve(stim, np.ones((n_block, 1)), axes=0)[:n_times]
    dm_events = pd.DataFrame({
        'trial_type': [conditions[ii] for ii in idx[1]],
        'onset': idx[0] / raw_h_long.info['sfreq'],
        'duration': n_block / raw_h_long.info['sfreq']})
    dm = make_first_level_design_matrix(
        raw_h_long.times, dm_events, hrf_model='glover',
        drift_model='polynomial', drift_order=0)
        
    return stim, dm, events

print("All done!") 

All done!


In [ ]:

# To skip above block, just run this:
#df_cha = pd.read_csv(op.join(results_path, 'df_cha_theta.csv'))


In [36]:
# Block averages
event_id = {condition: ci for ci, condition in enumerate(conditions, 1)}
evokeds = {condition: dict() for condition in conditions}
for day in days:
    for subject in subjects:
        fname = op.join(proc_path, f'{subject}_{day}_{exp_name}-ave.fif')
#        if not op.isfile(fname):
        tmin, tmax = -2, 38
        baseline = (None, 0)
        t0 = time.time()
        print(f'Creating block average for {subject} day {day}... ', end='')
        raws = list()
        events = list()
        for run in runs:
            fname2 = op.join(proc_path, f'{subject}_{day}_{run:03d}_long_hbo_raw.fif')
            raw_h = mne.io.read_raw_fif(fname2)
            events.append(_make_design(raw_h, None, 'block', subject, run)[2])
            raws.append(raw_h)
        bads = sorted(set(sum((r.info['bads'] for r in raws), [])))
        for r in raws:
            r.info['bads'] = bads
        raw_h, events = mne.concatenate_raws(raws, events_list=events)
        epochs = mne.Epochs(raw_h, events, event_id, tmin=tmin, tmax=tmax,
                            baseline=baseline)
        this_ev = [epochs[condition].average() for condition in conditions]
        assert all(ev.nave > 0 for ev in this_ev)
        mne.write_evokeds(fname, this_ev, overwrite=True)
        print(f'{time.time() - t0:0.1f} sec')
        for condition in conditions:
            evokeds[condition][subject] = mne.read_evokeds(fname, condition)
        print(f'Done for {group} {subject} day {day} run {run:03d}... ', end='')


# Mark bad channels
bad = dict()
bb = dict()

for day in days:
    for subject in subjects:
        for run in runs:
            fname2 = op.join(proc_path, f'{subject}_{day}_{run:03d}_long_hbo_raw.fif')
            this_info = mne.io.read_info(fname2)
            bad_channels = [idx - 1 for idx in sorted(
                this_info['ch_names'].index(bad) + 1 for bad in this_info['bads'])]
            valid_indices = np.arange(len(use.ch_names))
            bb = [b for b in bad_channels if b in valid_indices]
            bad[(subject, run, day)] = bb
        assert np.in1d(bad[(subject, run, day)], np.arange(len(use.ch_names))).all()  # noqa: E501

bad_combo = dict()
for day in days:
    for (subject, run, day), bb in bad_channels:
        bad_combo[subject] = sorted(set(bad_combo.get(subject, [])) | set(bb))
bad = bad_combo
#assert set(bad) == set(subjects)

start = len(df_cha)
n_drop = 0
for day in days:
    for (subject, run, day), bb in bad_channels:
        if not len(bb):
            continue
        drop_names = [use.ch_names[b] for b in bb]
        is_subject = (df_cha['subject'] == subject)
        is_day = (df_cha['day'] == day)
        assert len(is_subject) == len(df_cha)
        is_day = (df_cha['day'] == day)
        drop = df_cha.index[
            is_subject &
            is_day &
            np.in1d(df_cha['ch_name'], drop_names)]
        n_drop += len(drop)
        if len(drop):
            print(f'Dropping {len(drop)} for {subject} day {day}')  # {run}')
            df_cha.drop(drop, inplace=True)
end = len(df_cha)
assert n_drop == start - end, (n_drop, start - end)

# Combine runs by averaging
sorts = ['subject', 'ch_name', 'Chroma', 'Condition', 'group', 'day', 'run']
df_cha.sort_values(
    sorts, inplace=True)
theta = np.array(df_cha['theta']).reshape(-1, len(runs)).mean(-1)
df_cha.drop(
    [col for col in df_cha.columns if col not in sorts[:-1]], axis='columns',
    inplace=True)
df_cha.reset_index(drop=True, inplace=True)
df_cha = df_cha[::len(runs)]
df_cha.reset_index(drop=True, inplace=True)
df_cha['theta'] = theta
df_cha.to_csv(op.join(results_path, 'df_cha_theta_tddr_bp.csv'), index=False)

print("All done!") 

Creating block average for 201 day 1... Opening raw data file ../../processed/201_1_001_long_hbo_raw.fif...
    Range : 0 ... 3994 =      0.000 ...   830.752 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file ../../processed/201_1_002_long_hbo_raw.fif...
    Range : 0 ... 3838 =      0.000 ...   798.304 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Not setting metadata
200 matching events found
Setting baseline interval to [-2.0800001110839905, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Overwriting existing file.
3.8 sec
Reading /Users/ansle/Documents/GitHub/fnirs-analysis/GithubCode/fnirs-analysis/../../processed/201_1_av-ave.fif ...
    Found the data of interest:
        t =   -2080.00 ...   38064.00 ms (A)
        0 CTF compensation matrices available
        nave = 50 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add

In [ ]:
# THIS IS FOR CHANNEL x CONDITION ANALYSIS ONLY

# Mixed linear model
def _mixed_df(ch_summary):
    formula = "theta ~ -1 + ch_name:Condition" 
    ch_model = smf.mixedlm(
        formula, ch_summary, groups=ch_summary["subject"]).fit(method='powell')
    ch_model_df = mne_nirs.statistics.statsmodels_to_results(ch_model)
    ch_model_df['P>|z|'] = ch_model.pvalues
    ch_model_df.drop([idx for idx in ch_model_df.index if '[constant]' in idx],
                    inplace=True)
    return ch_model_df

print("All done!") 

# Make separate subject lists for trained and untrained (TEST)
#trained_subjects = {'201 203 207 212 213 214 216 219 221 223'}
#control_subjects = {'202 204 205 206 208 209 215 217 218 224'}

# Run group level model and convert to dataframe
ch_summary = df_cha.query("Chroma in ['hbo']").copy()   ### ALSO RUN HBR ANALYSIS
#ch_summary = df_cha.query("Chroma in ['hbo'] and group in ['trained'] and day in ['3']").copy()
ch_model_df = _mixed_df(ch_summary) 
ch_model_df.reset_index(inplace=True)

# Correct for multiple comparisons
print(f'Correcting for {len(ch_model_df["P>|z|"])} comparisons using FDR')
_, ch_model_df['P_fdr'] = mne.stats.fdr_correction(
    ch_model_df['P>|z|'], method='indep')
ch_model_df['SIG'] = ch_model_df['P_fdr'] < 0.05
ch_model_df.to_csv(op.join(results_path, 'ch_model_corrected_hbo.csv'), index=False)
ch_model_df.loc[ch_model_df.SIG == True]

print("All done!") 

In [ ]:
# ###############################################################################
# # Plot significant channels

# sig_chs = dict()
# zs = dict()
# for condition in conditions:
#     sig_df = ch_model_df[
#         (ch_model_df['P_fdr'] < 0.05) &
#         (ch_model_df['Condition'] == condition) &
#         (ch_model_df['ch_name'].isin(use.ch_names))
#         ]
#     sig_chs[(condition)] = sorted(
#         (use.ch_names.index(row[1]['ch_name']), row[1]['P_fdr'])
#         for row in sig_df.iterrows())
#     ch_model_df[ch_model_df['Condition'] == condition]
#     zs[condition] = np.array([
#         ch_model_df.loc[(ch_model_df['Condition'] == condition) & 
#                         (ch_model_df['ch_name'] == ch_name), 'z'].iloc[0]
#         for ch_name in info['ch_names'][::2]], float)
#     assert zs[condition].shape == (84,)
#     assert np.isfinite(zs[condition]).all()

# def _plot_sig_chs(sigs, ax):
#     if sigs and isinstance(sigs[0], tuple):
#         sigs = [s[0] for s in sigs]
#     ch_groups = [sigs, np.setdiff1d(np.arange(info['nchan']), sigs)]
#     mne.viz.plot_sensors(
#         info, 'topomap', 'hbo', title='', axes=ax,
#         show_names=True, ch_groups=ch_groups)
#     ax.collections[0].set(lw=0)
#     c = ax.collections[0].get_facecolor()
#     c[(c[:, :3] == (0.5, 0, 0)).all(-1)] = (0., 0., 0., 0.1)
#     c[(c[:, :3] == (0, 0, 0.5)).all(-1)] = (0., 1., 0., 0.5)
#     ax.collections[0].set_facecolor(c)
#     ch_names = [info['ch_names'][idx] for idx in sigs]
#     texts = list(ax.texts)
#     got = []
#     for text in list(texts):
#         try:
#             idx = ch_names.index(text.get_text())
#         except ValueError:
#             text.remove()
#         else:
#             got.append(idx)
#             text.set_text(f'{sigs[idx] // 2 + 1}')
#             text.set(fontsize='xx-small', zorder=5, ha='center')
#     assert len(got) == len(sigs), (got, list(sigs))

# def _plot_sigs(sig_chs, all_corrs=()):
#     n_col = max(len(x) for x in sig_chs.values()) + 1
#     n_row = len(conditions)
#     figsize = (n_col * 1.0, n_row * 1.0)
#     fig, axes = plt.subplots(
#         n_row, n_col, figsize=figsize, constrained_layout=True, squeeze=False)
#     h_colors = {0: 'r', 1: 'b'}
#     xticks = [0, 10, 20, 30]
#     ylim = [-0.2, 0.3]
#     yticks = [-0.2, -0.1, 0, 0.1, 0.2, 0.3]
#     xlim = [times[0], 35]
#     ylim = np.array(ylim)
#     yticks = np.array(yticks)
#     for ci, condition in enumerate(conditions):
#         ii = 0
#         sigs = sig_chs[condition]
#         if len(sigs) == 0:
#             sigs = [(None, None)]
#         for ii, (ch_idx, ch_p) in enumerate(sigs):
#             ax = axes[ci, ii]
#             if ch_idx is not None:
#                 for jj in range(2):  # HbO, HbR
#                     color = h_colors[jj]
#                     a = 1e6 * np.array(
#                         [evokeds[condition][subject].data[ch_idx + jj]
#                          for subject in use_subjects
#                          if ch_idx + jj not in bad.get(subject, [])], float)
#                     m = np.mean(a, axis=0)
#                     lower, upper = stats.t.interval(
#                         0.95, len(a) - 1, loc=m, scale=stats.sem(a, axis=0))
#                     ax.fill_between(
#                         times, lower, upper, facecolor=color,
#                         edgecolor='none', lw=0, alpha=0.25, zorder=3,
#                         clip_on=False)
#                     ax.plot(times, m, color=color, lw=1, zorder=4,
#                             clip_on=False)
#                 # Correlations
#                 this_df = ch_summary_use.query(
#                     f'ch_name == {repr(use.ch_names[ch_idx])} and '
#                     f'Chroma == "hbo" and '
#                     f'Condition == {repr(condition)}')
#                 #assert 8 <= len(this_df) <= len(subjects), len(this_df)
#                 a = np.array(this_df['theta'])
#                 cs = list()
#                 if len(cs):
#                     cs = [''] + cs
#                 c = '\n'.join(cs)
#                 ax.text(times[-1], ylim[1],
#                         f'ch{ch_idx // 2 + 1}\np={ch_p:0.5f}{c}',
#                         ha='right', va='top', fontsize='x-small')
#             ax.axvline(20, ls=':', color='0.5', zorder=2, lw=1)
#             ax.axhline(0, ls='-', color='k', zorder=2, lw=0.5)
#             ax.set(xticks=xticks, yticks=yticks)
#             ax.set(xlim=xlim, ylim=ylim)
#             for key in ('top', 'right'):
#                 ax.spines[key].set_visible(False)
#             if ax.get_subplotspec().is_last_row():
#                 ax.set(xlabel='Time (sec)')
#             else:
#                 ax.set_xticklabels([''] * len(xticks))
#             if ax.get_subplotspec().is_first_col():
#                 ax.set_ylabel(condition)
#             else:
#                 ax.set_yticklabels([''] * len(yticks))
#             for key in ('top', 'right'):
#                 ax.spines[key].set_visible(False)
#         for ii in range(ii + 1, n_col - 1):
#             fig.delaxes(axes[ci, ii])
#         # montage
#         ax = axes[ci, -1]
#         if sigs[0][0] is None:
#             fig.delaxes(ax)
#         else:
#             # plot montage
#             _plot_sig_chs(sigs, ax)
#     return fig

# times = evokeds[conditions[0]][subjects[0]].times
# info = evokeds[conditions[0]][subjects[0]].info
# fig = _plot_sigs(sig_chs)
# for ext in ('png', 'svg'):
#     fig.savefig(op.join(results_path, f'stats_{exp_name}.{ext}'))

# print("All done!") 


In [ ]:
# # ONLY PLOT SENSORS

# def _plot_sigs(sig_chs, all_corrs=()):
#     n_col = 1  # Only need one column for sensor locations
#     n_row = len(conditions)
#     figsize = (n_col * 2, n_row * 2)  # Increase figure size for better label visibility
#     fig, axes = plt.subplots(
#         n_row, n_col, figsize=figsize, constrained_layout=True)

#     # Handle the case of a single subplot
#     if n_row == 1:
#         axes = [axes]

#     for ci, condition in enumerate(conditions):
#         sigs = sig_chs[condition]
#         ax = axes[ci]  # Direct reference to each subplot's axis
        
#         # Ensure labels are set even if there are no significant sensors
#         ax.set_ylabel(condition, labelpad=100)  # Increase labelpad if necessary

#         # Only attempt to plot sensor locations if there are significant sensors
#         if sigs[0][0] is not None:
#             _plot_sig_chs(sigs, ax)
#         else:
#             # Optionally, clear the axes but keep them to display the label
#             ax.clear()  # Clear the axes of any plotted data
#             ax.set_xticks([])  # Remove x-ticks
#             ax.set_yticks([])  # Remove y-ticks
#             ax.spines['top'].set_visible(False)
#             ax.spines['right'].set_visible(False)
#             ax.spines['bottom'].set_visible(False)
#             ax.spines['left'].set_visible(False)
#     return fig

# times = evokeds[conditions[0]][subjects[8]].times
# info = evokeds[conditions[0]][subjects[8]].info
# fig = _plot_sigs(sig_chs)

# for ext in ('png', 'svg'):
#     fig.savefig(op.join(results_path, f'sensors_{exp_name}.{ext}'))

# print("All done!") 

In [ ]:
# ###############################################################################
# # Source space projection
# import pyvista

# info = use.copy().pick_types(fnirs='hbo', exclude=()).info
# info['bads'] = []
# assert tuple(zs) == conditions

# evoked = mne.EvokedArray(np.array(list(zs.values())).T, info)
# picks = np.arange(len(evoked.ch_names))

# for ch in evoked.info['chs']:
#     assert ch['coord_frame'] == mne.io.constants.FIFF.FIFFV_COORD_HEAD
# stc = mne.stc_near_sensors(
#     evoked, trans='fsaverage', subject='fsaverage', mode='weighted',
#     distance=0.02, project=True, picks=picks, subjects_dir=subjects_dir)
# # Split channel indices by left lat, posterior, right lat:
# num_map = {name: str(ii) for ii, name in enumerate(evoked.ch_names)}
# evoked.copy().rename_channels(num_map) #.plot_sensors(show_names=True)
# view_map = [np.arange(19), np.arange(19, 33), np.arange(33, 52)]
# surf = mne.read_bem_surfaces(  # brain surface
#     f'{subjects_dir}/fsaverage/bem/fsaverage-5120-5120-5120-bem.fif', s_id=1)

# for ci, condition in enumerate(conditions):
#     this_sig = [v[0] // 2 for v in sig_chs[condition]]
#     #assert np.in1d(this_sig, np.arange(52)).all()
#     pos = np.array([info['chs'][idx]['loc'][:3] for idx in this_sig])
#     pos.shape = (-1, 3)  # can be empty
#     trans = mne.transforms._get_trans('fsaverage', 'head', 'mri')[0]
#     pos = mne.transforms.apply_trans(trans, pos)  # now in MRI coords
#     pos = mne.surface._project_onto_surface(pos, surf, project_rrs=True)[2]
#     # plot
#     brain = stc.plot(hemi='both', views=['lat', 'frontal', 'lat'],
#                     initial_time=evoked.times[ci], cortex='low_contrast',
#                     time_viewer=False, show_traces=False,
#                     surface='pial', smoothing_steps=0, size=(1200, 400),
#                     clim=dict(kind='value', pos_lims=[0., 1.25, 2.5]),
#                     colormap='RdBu_r', view_layout='horizontal',
#                     colorbar=(0, 1), time_label='', background='w',
#                     brain_kwargs=dict(units='m'),
#                     add_data_kwargs=dict(colorbar_kwargs=dict(
#                         title_font_size=24, label_font_size=24, n_labels=5,
#                         title='z score')), subjects_dir=subjects_dir)
#     brain.show_view('lat', hemi='lh', row=0, col=0)
#     brain.show_view(azimuth=270, elevation=90, row=0, col=1)
#     pl = brain.plotter
#     used = np.zeros(len(this_sig))
#     brain.show_view('lat', hemi='rh', row=0, col=2)
#     plt.imsave(
#         op.join(results_path, f'all_brain_{exp_name}_{condition}.png'), pl.image)

# print("All done!") 

In [ ]:
# vtk==9.1.0 --> redid with conda --> 9.0.3 --> 9.0.3 ??
# pyvistaqt==0.2.0 --> upgrade --> 0.11.1
# pyvista==0.32.0 --> redid with pip --> 0.44.1
# numpy 1.26.4 --> 1.19.5 --> 1.23.0
# pip install vtk==9.1.0 --no-cache-dir
# "pip show X" to see version // conda list vtk
# conda install -c conda-forge vtk

In [ ]:
# fOLD specificity
import xlrd

fold_files = ['10-10.xls', '10-5.xls']
for fname in fold_files:
    if not op.isfile(fname):
        pooch.retrieve(f'https://github.com/nirx/fOLD-public/raw/master/Supplementary/{fname}', None, fname, path=os.getcwd())  # noqa
raw_spec = use.copy()
raw_spec.pick_channels(raw_spec.ch_names[::2])
specs = mne_nirs.io.fold_channel_specificity(raw_spec, fold_files, 'Brodmann')
for si, spec in enumerate(specs, 1):
    spec['Channel'] = si
    spec['negspec'] = -spec['Specificity']
specs = pd.concat(specs, ignore_index=True)
specs.drop(['Source', 'Detector', 'Distance (mm)', 'brainSens',
            'X (mm)', 'Y (mm)', 'Z (mm)'], axis=1, inplace=True)
specs.sort_values(['Channel', 'negspec'], inplace=True)
specs.drop('negspec', axis=1, inplace=True)
specs.reset_index(inplace=True, drop=True)
specs.to_csv(op.join(results_path, 'specificity.csv'), index=False)

In [38]:
# CALCULATE HbDIFF

# Load the data
df_cha = pd.read_csv(op.join(results_path, 'df_cha_theta_tddr_bp_detrend.csv'))
print(df_cha)
df_cha_nolabels = df_cha.copy()
df_cha_nolabels['ch_name'] = df_cha_nolabels['ch_name'].str[:-4]

# Separate HbO and HbR
df_hbo = df_cha_nolabels[df_cha_nolabels['Chroma'].str.endswith('hbo')].set_index(['subject', 'Condition', 'group', 'day', 'ch_name']).sort_index()
df_hbr = df_cha_nolabels[df_cha_nolabels['Chroma'].str.endswith('hbr')].set_index(['subject', 'Condition', 'group', 'day', 'ch_name']).sort_index()

# Compute the difference
df_cha_diff_list = []
for ch_name in df_hbo.index.get_level_values('ch_name').unique():
    # Get aligned indices
    df_hbo_ch = df_hbo.loc[(slice(None), slice(None), slice(None), slice(None), ch_name), :].sort_index()
    df_hbr_ch = df_hbr.loc[(slice(None), slice(None), slice(None), slice(None), ch_name), :].sort_index()
    
    # Ensure df_hbo_ch and df_hbr_ch have the same length
    common_index = df_hbo_ch.index.intersection(df_hbr_ch.index)
    df_hbo_ch = df_hbo_ch.loc[common_index]
    df_hbr_ch = df_hbr_ch.loc[common_index]
    
    # Calculate the difference
    df_diff = df_hbo_ch[['theta']].sub(df_hbr_ch[['theta']])
    
    # Align df_cha_ch with df_diff
    df_cha_ch = df_hbo_ch.reset_index()
    df_cha_ch['theta'] = df_diff.values
    df_cha_ch['Chroma'] = 'hbdiff'
    df_cha_ch['ch_name'] = df_cha_ch['ch_name'] + ' hbdiff'
    
    if not df_cha_ch.empty:
        df_cha_diff_list.append(df_cha_ch)

df_cha_diff_concat = pd.concat(df_cha_diff_list, ignore_index=True)

# Concatenate original df_cha with df_cha_diff_concat
df_final = pd.concat([df_cha, df_cha_diff_concat], ignore_index=True)
df_final.to_csv(op.join(results_path, 'df_combined_final_cha_tddr_bp.csv'), index=False)

# Print the head of the resulting dataframe
print(df_final)
print("All done!")

      Condition Chroma      ch_name  subject  day    group         theta
0             A    hbo  S10_D18 hbo      201    1  trained -1.577924e-05
1             A    hbo  S10_D18 hbo      201    3  trained  3.961849e-06
2            AV    hbo  S10_D18 hbo      201    1  trained  1.228112e-06
3            AV    hbo  S10_D18 hbo      201    3  trained -5.628723e-06
4             V    hbo  S10_D18 hbo      201    1  trained -7.734326e-06
...         ...    ...          ...      ...  ...      ...           ...
33595         V    hbr   S9_D21 hbr      234    3  control  5.178583e-06
33596         W    hbr   S9_D21 hbr      234    1  control -1.054386e-06
33597         W    hbr   S9_D21 hbr      234    3  control -5.320285e-06
33598  constant    hbr   S9_D21 hbr      234    1  control -6.760430e-08
33599  constant    hbr   S9_D21 hbr      234    3  control  6.881836e-07

[33600 rows x 7 columns]
      Condition  Chroma        ch_name  subject  day    group         theta
0             A     hb

In [ ]:
# RUN T-TEST AND PLOT THE CHANGES OVER TIME

# Source space projection - significant changes over time
import pandas as pd
import mne
import statsmodels.api as sm
from scipy.stats import ttest_rel, zscore
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from io import BytesIO

raw_path = '../../data'
proc_path = '../../processed'
results_path = '../../results'
subjects_dir = '../../subjects'

groups = ['trained', 'control']
chromas = ['hbo', 'hbr', 'hbdiff']
conditions = ['A', 'V', 'AV']

df_final = pd.read_csv(op.join(results_path, 'df_combined_final_cha_sdd.csv'))

# Perform analysis for each group and Chroma
for group in groups:
    for chroma in chromas:
        # Prepare figure for composite plots
        fig, axes = plt.subplots(1, len(conditions), figsize=(15, 5))
        for idx, condition in enumerate(conditions):
            # Filter data for day 1 and day 3 for the specific group and Chroma
            df_day1 = df_final.query(f"group == '{group}' and Chroma == '{chroma}' and day == 1").copy()
            df_day3 = df_final.query(f"group == '{group}' and Chroma == '{chroma}' and day == 3").copy()

            # Set index and sort
            df_day1 = df_day1.set_index(['subject', 'ch_name', 'Condition', 'Chroma']).sort_index()
            df_day3 = df_day3.set_index(['subject', 'ch_name', 'Condition', 'Chroma']).sort_index()
            
            # Merge dataframes to align day 1 and day 3 data
            df_merged = df_day1[['theta']].rename(columns={'theta': 'theta_day1'}).merge(
                df_day3[['theta']].rename(columns={'theta': 'theta_day3'}),
                left_index=True, right_index=True)

            # Calculate the difference and z-score
            df_merged['theta_diff'] = df_merged['theta_day3'] - df_merged['theta_day1']
            df_merged['z'] = zscore(df_merged['theta_diff'])

            # Perform paired t-test for each channel and condition across subjects
            t_stats = []
            p_values = []
            ch_names = []
            condition_list = []

            for (ch_name, cond), group_df in df_merged.groupby(['ch_name', 'Condition']):
                t_stat, p_value = ttest_rel(group_df['theta_day1'], group_df['theta_day3'])
                t_stats.append(t_stat)
                p_values.append(p_value)
                ch_names.append(ch_name)
                condition_list.append(cond)

            # Create a results DataFrame
            results_df = pd.DataFrame({
                'ch_name': ch_names,
                'Condition': condition_list,
                't_stat': t_stats,
                'p_value': p_values
            })

            # Combine with z-score data
            z_scores = df_merged.groupby(['ch_name', 'Condition'])['z'].mean().reset_index()
            results_df = results_df.merge(z_scores, on=['ch_name', 'Condition'])

            # Correct for multiple comparisons
            print(f'Correcting for {len(results_df["p_value"])} comparisons using FDR')
            _, results_df['P_fdr'] = mne.stats.fdr_correction(results_df['p_value'], method='indep')
            results_df['SIG'] = results_df['p_value'] < 0.05
            
            # Print significant results
            significant_results = results_df.loc[results_df.SIG == True]
            print(significant_results)

            # Prepare data for brain plots
            fname = op.join(proc_path, f'205_1_001_hbo_raw.fif')
            use = mne.io.read_raw_fif(fname)
            info = use.info

            ch_of_interest = use.pick_channels([ch_name for ch_name in use.info['ch_names']])
            info_of_interest = ch_of_interest.info

            zs = {}
            condition_data = results_df[(results_df['Condition'] == condition) & (results_df['SIG'] == True)]
            
            # Debugging prints
            print(f"\nCondition: {condition}")
            print(f"Group: {group}")
            print(f"Chroma: {chroma}")
            print(f"Condition Data:\n{condition_data.head()}")
            
            zs[condition] = np.array([
                condition_data.loc[(condition_data['ch_name'] == ch_name) & (condition_data['SIG'] == True), 'z'].values[0]
                if not condition_data.loc[(condition_data['ch_name'] == ch_name) & (condition_data['SIG'] == True), 'z'].empty
                else 0
                for ch_name in info_of_interest['ch_names']
            ])
            
            # Create an EvokedArray for each condition
            evoked = mne.EvokedArray(zs[condition][:, np.newaxis], info_of_interest)
            picks = np.arange(len(info_of_interest['ch_names']))

            stc = mne.stc_near_sensors(
                evoked, trans='fsaverage', subject='fsaverage', mode='weighted',
                distance=0.02, project=True, picks=picks, subjects_dir=subjects_dir)

            # Plot the brain and capture the image in-memory
            brain = stc.plot(hemi='both', views=['lat', 'frontal', 'lat'],
                             cortex='low_contrast', time_viewer=False, show_traces=False,
                             surface='pial', smoothing_steps=0, size=(1200, 400),
                             clim=dict(kind='value', pos_lims=[0, 0.5, 1]),
                             colormap='RdBu_r', view_layout='horizontal',
                             colorbar=(0, 1), time_label='', background='w',
                             brain_kwargs=dict(units='m'),
                             add_data_kwargs=dict(colorbar_kwargs=dict(
                                 title_font_size=24, label_font_size=20, n_labels=5,
                                 title='z score')), subjects_dir=subjects_dir)
            brain.show_view('lat', hemi='lh', row=0, col=0)
            brain.show_view(azimuth=270, elevation=90, row=0, col=1)
            brain.show_view('lat', hemi='rh', row=0, col=2)

            # Capture the plot as an image in memory
            screenshot = brain.screenshot(time_viewer=False)
            brain.close()

            # Display the image in the composite figure
            ax = axes[idx]
            ax.imshow(screenshot)
            ax.axis('off')
            ax.set_title(f'{group.capitalize()} - Condition {condition} ({chroma})', fontsize=18)

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.savefig(op.join(results_path, f'{group}_{chroma}_composite_brain_plots_bp.png'))
        plt.show()

print("All done!")